# Naive Bayes Spam Filter
In this notebook we will lean about, train and validate a Naive Bayes email spam filter (classifier). The plan is to develop the classifier and then train and validate it on the [UCI spambase dataset](https://archive.ics.uci.edu/ml/datasets/Spambase) to determine what messages are spam or not spam. 

## Bayes Theorem

Given the event that the message is spam by S and the message contains a word by W, Bayes theorem allows us to calculate the probability that the message is spam given the word(s) from the trained probabilities of the message is spam if it contains a word. 

$$ 
P(S|W) = \frac{P(W|S)P(S)}{P(W)} = \frac{P(W|S)P(S)}{P(W|S)P(S) + P(W|!S)P(!S)} 
$$

*Example*: Assume that spam and non spam messages are equally likely - P(S) = P(!S) = 0.5. Then also assume tha the word 'viagra' appears in 50% of spam emails and 1% in non-spam emails. Then Bayes theorem implies that the probability that a given email containing viagra is spam is

$$
P(S|viagra) = \frac{P(viagra|S)}{P(viagra|S) + P(viagra|!S)} = \frac{0.5}{0.5 + 0.01} = 98 \%.
$$

This ML algorithm is naive because the probabiltities of words being in spam or ham messages are independent.

The goal of this alogirthm is to calculate P(W|S) and P(W|!S) and we will do this in the following steps
- Load the data. This UCI dataset contains 48 columns with occurance frequenices of 48 different words and the last column is the target (spam or ham). Use the load_data() function
- Split the data into a training and validation subsets Use the split_data() function
- For each of the 48 words, calculate the number of spam and ham emails that word is found in. Use the count_spams_hams() function
- Calculate the liklihood that a message is spam or ham for each word using the word_probabilities() function
- Write a function that takes in the likelihoods and a message and uses Bayes theorem to calculate P(S|W) (where W is a set of words in the message) and classifies the message as spam or ham

In [1]:
import pandas as pd
import os
import numpy as np
import collections

import sklearn.metrics
import sklearn.naive_bayes

import get_words

np.random.seed(123) # So you can exactly reproduce my numbers.

In [2]:
word_catagories = np.array(get_words.get_words())

def load_data():
    """ 
    Load the spambase dataset
    """
    column_names = np.concatenate((word_catagories, ['target']))
    column_numbers = np.concatenate((np.arange(len(word_catagories)), [57]))

    data = pd.read_csv(os.path.join('.', 'data', 'spambase.data'), 
                       names=column_names, usecols=column_numbers)
    #data[word_catagories] *= 10000 # Scaling factor to go from percentage to number of words 
    return data

def split_data(data, p):
    """
    Split the data according to the probability p of picking a training sample.
    Returns data with the same number of columns, except the first DataFrame 
    (training set) has p*data.shape[0] number of rows and the second DataFrame 
    (validation set) has (1-p)*data.shape[0] number of rows.
    """ 
    n_samples = data.shape[0]
    n_train = int(n_samples * p)
    idt = np.random.choice(np.arange(n_samples), size=n_train, replace=False)
    idv = np.array([i for i in range(n_samples) if i not in idt])
    return data.loc[idt], data.loc[idv]

Load the spambase dataset, and split it into a train and validate subsets.

In [3]:
data = load_data()
train, validate = split_data(data, 0.5)

In [4]:
train.head()

,make,address,all,our,over,remove,internet,order,mail,receive,...,direct,cs,meeting,original,project,re,edu,table,conference,target
2786,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0
463,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.08,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,1
3269,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.89,0.0,0.0,0.0,0.89,0
380,0.6,0.0,0.36,0.0,1.44,0.0,0.0,0.00,0.24,1.32,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,1
3118,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0


For each word, we now tally up the number of spam and non-spam messages that word was in.

In [5]:
def count_spams_hams(train):
    """
    Counts the number of messages that contain at least one
    instence (non-zero probability) of each word. 
    """
    # key is the word and value is an integer value of how many
    # spam or ham messages this word was in
    spam_counts = collections.defaultdict(int)
    ham_counts = collections.defaultdict(int)

    # loop over the messages in the training set
    for _, message in train.iterrows():
        spam_status = message['target']

        # Loop over all word_categories and check if any of them were 
        # in a spam or non-spam message (had a non-zero probability)
        for i, word in enumerate(word_catagories):
            # if spam and word is in the message
            if (message[i] > 0) and (spam_status): 
                # Add the number of times word occured in that message
                spam_counts[word] += 1
            # if non-spam message contains word
            elif (message[i] > 0) and (not spam_status): 
                ham_counts[word] += 1
    return spam_counts, ham_counts

spam_counts, ham_counts = count_spams_hams(train)

Calculate the likelihoods that a message is spam or not spam given each word (train the Naive Bayes classifier). 

P(w|S) = (k + numer of spams containing word w)\(2k + numer of spams)

In [6]:
def word_probabilities(train, spam_counts, ham_counts, k=1):
    """
    Calculates the likelihood of word given spam and word given not spam.
    Returns a tuple of (word, p(word|spam), p(word|!spam)). k is the 
    smoothing factor
    """
    n_spams = sum(train.target) # total number of spam messages 
    n_hams = train.shape[0] - n_spams
    
    t = [(word, 
         (k + spam_counts[word])/(k + n_spams),
         (k + ham_counts[word])/(k + n_hams)
         ) for word in spam_counts.keys()]
    return t

word_likelihood = word_probabilities(train, spam_counts, ham_counts)

This function takes in the trained word_likelihoods and calculates the probability that the message is spam.

In [7]:
def spam_probability(word_likelihoods, message):
    """
    Calculate the probability that the message is spam, given the
    word likelihoods. We add the log of the probabilities to avoid
    a numerical underflow error.
    """
    log_prob_spam = log_prob_ham = 0.0
    
    for word, spam_like, ham_like in word_likelihoods:
        # Find the index of the probability in message that
        # corresponds to the likelihood we are currently comparing
        idx = np.where(word == word_catagories)[0]
        assert len(idx) == 1, f"One or too many matches! idx={idx}"
        
        # If word was found in the message.
        if message[idx[0]] > 0:
            log_prob_spam += np.log(spam_like)
            log_prob_ham += np.log(ham_like)
        # If the word was not found, add the probability of not seeing it.
        else:
            log_prob_spam += np.log(1-spam_like)
            log_prob_ham += np.log(1-ham_like)
            
    prob_spam = np.exp(log_prob_spam)
    prob_ham = np.exp(log_prob_ham)
    return prob_spam / (prob_spam + prob_ham)        

Validate the classifier using the validation dataset.

In [8]:
filter_targets = np.zeros(validate.shape[0])

for i, (_, row) in enumerate(validate.iterrows()):
    p_i = spam_probability(word_likelihood, row)
    if p_i > 0.5:
        filter_targets[i] = 1
    else:
        filter_targets[i] = 0

What is the confusion matrix for this model?

In [9]:
confusion = sklearn.metrics.confusion_matrix(validate.target, filter_targets)
print(confusion, '=\n\n', np.array([['TP', 'FP'], ['FN', 'TN']]), 
      '\n FP = type 1 error | FN = type 2 error',
      '\n(rows: pedicted spam and ham. columns: actual spam and ham)')

[[1260  140]
 [ 176  725]] =

 [['TP' 'FP']
 ['FN' 'TN']] 
 FP = type 1 error | FN = type 2 error 
(rows: pedicted spam and ham. columns: actual spam and ham)


And accuracy?

In [10]:
print(round(100*(confusion[0, 0] + confusion[1, 1])/np.sum(confusion)), 
      '% of messages were correctly classified as spam or not spam')

86.0 % of messages were correctly classified as spam or not spam


Not bad for such a "naive" classifier! We can do better with more data (spambase.data has other columns that I did not use here), tweak the smoothing (k) factor, and add a fraction threshold to consider that word in a spam or ham message. 

Now lets find the spammiest and hammiest words - P(S|W).

In [16]:
def p_spam_given_word(word_like):
    """
    Calculate P(S|W) using Bayes theorem for all words in the likelihoods tuples. 
    """
    word, p_if_spam, p_if_ham = word_like
    p = p_if_spam/(p_if_spam + p_if_ham)
    return p

word_p = sorted(word_likelihood, key=p_spam_given_word)
print(f'Spammiest words: {word_p[-5:]}')
print(f'Hammiest words: {word_p[:5]}')

Spammiest words: [('money', 0.05585980284775466, 0.005039596832253419), ('hp', 0.3362541073384447, 0.024478041756659467), ('your', 0.20262869660460023, 0.014398848092152628), ('hpl', 0.38116100766703176, 0.014398848092152628), ('internet', 0.411829134720701, 0.012239020878329733)]
Hammiest words: [('labs', 0.007667031763417305, 0.28005759539236863), ('pm', 0.002190580503833516, 0.07919366450683946), ('conference', 0.002190580503833516, 0.051115910727141826), ('lab', 0.013143483023001095, 0.29301655867530596), ('george', 0.019715224534501644, 0.3779697624190065)]


## Tasks: 
- Use a library (e.g. scikit-learn) to classify and validate this dataset. *Hint* use sklearn.naive_bayes.BernoulliNB()